<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'> Business Loyalty Agent: Built with Teradata AgentStack (pro-code) leveraging Teradata Enterprise Vector Store
      
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style="font-size:20px;font-family:Arial"><b>Introduction:</b></p>

<p style="font-size:16px;font-family:Arial">
In many finance and compliance workflows, the hardest questions aren't solved using SQL alone.
Eligibility rules, policies, and exceptions usually live in unstructured documents, while the facts needed to
evaluate those rules live in structured tables.
</p>

<p style="font-size:16px; font-family:Arial">
This notebook demonstrates how easy it is to build a <b>finance loyalty agent</b> using
<b>Teradata package for LangChain, Teradata Vector Store, and the Teradata MCP</b>. The agent answers
real business questions by combining:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>Unstructured policy definitions</b> stored in a Teradata Vector Store</li>
  <li><b>Structured business data</b> stored in Teradata tables</li>
  <li><b>Pre-approved, governed SQL logic</b> exposed through MCP tools</li>
</ul>

<p style="font-size:16px; font-family:Arial">
Rather than generating SQL or relying on assumptions, the agent is limited
to use only trusted data sources and approved SQL code, which make it suitable for regulated
finance and compliance-sensitive workflows.
</p>

<div style="text-align:center">
  <img src="./images/agent_architecture.png"
       width="900"
       alt="Agent Architecture"
       style="border:4px solid #404040; border-radius:10px;">
</div>

<p style="font-size:20px;font-family:Arial; margin-top:20px"><b>Business Value:</b></p>
<p style="font-size:16px;font-family:Arial">
The business question we want to answer is:
</p>
<p style="font-size:18px;font-family:Arial; font-style:italic; margin-left:20px;">
“Is a business eligible for a loyalty discount, and why?”
</p>

<div style="text-align:center; margin-top:15px; margin-bottom:20px">
  <img src="./images/agent_run_example.png"
       width="600"
       alt=" AI example"
       style="border:4px solid #404040; border-radius:10px;">
</div>
<p style="font-size:16px;font-family:Arial">
Answering this question requires:
</p>

<ul style="font-size:16px;font-family:Arial; margin-left:20px; line-height:1.8;">
  <li>Retrieving official policy and eligibility rules from documents</li>
  <li>Evaluating business facts such as tenure, status, region, and risk flags from structured tables</li>
  <li>Applying exceptions, exclusions, and effective dates defined in policy</li>
  <li>Producing an explainable answer suitable for audit and compliance review</li>
</ul>

<p style="font-size:16px;font-family:Arial"><b>This demo illustrates how Teradata enables:
:</b></p>

<ul style="font-size:16px;font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>Vector search</b> over the official policy documents using Teradata Vector Store via Teradata package for Langchain</li>
  <li><b>Governed SQL</b> execution, only approved MCP tools; no free-form SQL</li>
  <li><b>Agentic reasoning</b> that combines policy text, exceptions, exclusions, and dates with business data</li>
  <li><b>Governance by design</b> the agent never generates SQL and must justify every decision</li>
</ul>



<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>1. Configure the environment</b>

In [ ]:
%%capture
!pip install git+https://github.com/Teradata/teradata-mcp-server.git


In [ ]:
%%capture
!pip install -U langchain-teradata langchain-mcp-adapters langchain langchain-openai panel --quiet


<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Please</b><i> restart the kernel after executing the above cell to include/update these libraries into memory for this kernel. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i> and then clicking <b>Restart</b>.</p>
</div>

In [ ]:
import asyncio, sys, os
from getpass import getpass
from teradataml import *
from teradataml import create_context, set_auth_token
from teradataml import execute_sql
import ipywidgets as widgets
import asyncio
import panel as pn
from dotenv import load_dotenv
import pandas as pd
import time
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import HumanMessage


<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><i><b>Note:</b> To ensure that the Chatbot interface reflects the latest changes, please reload the page by clicking the <b>Reload</b> or <b>Refresh</b> button or pressing F5 on your keyboard for <b>first-time only</b> This will update the notebook with the latest modifications, and you'll be able to interact with the Chatbot using the new libraries.</i></p></div>

<hr style="height:2px;border:none">

<p style="font-size:20px;font-family:Arial">
  <b>2. Teradata Enterprise Vector Store</b>
</p>

<p style="font-size:16px;font-family:Arial">
Before creating our agent, we’ll configure and explore Teradata Enterprise Vector Store. 
We will create a vector store, load sample documents, and generate embeddings using the Teradata integration for LangChain.
To do this, we must first connect to VantageCloud and authenticate via UES.
    
</p>

<p style="font-size:16px;font-family:Arial">
<b>Connecting to Teradata:</b> The <code>create_context()</code> function establishes a connection to your Teradata Vantage system. 
This allows <code>teradataml</code> (and, by extension, the vector store) to authenticate and execute operations against the database.
</p>
<p style="font-size:16px;font-family:Arial">
All required credentials and the SSL certificate are provided in this notebook.
</p>
<p style="font-size:16px;font-family:Arial">
<b>Required parameters:</b><br>
• <b>host</b> – Your Teradata system address<br>
• <b>username / password</b> – Database credentials<br>
• <b>base_url</b> – UES API endpoint for your Teradata environment<br>
• <b>pat_token</b> – Personal Access Token for API authentication<br>
• <b>pem_file</b> – SSL certificate file for secure connections
    
</p>

<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Load the Environment Variables and Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Load the environment variables from a .env file and use them to create a connection context to VantageCloud.</p>


In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=Langchain_Teradata_MCP_Agent.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<p style = 'font-size:18px;font-family:Arial;'><b>2.2 Set your Authentication Token for this session.</b></p>


In [ ]:
# We've already loaded all the values into our environment variables and into a dictionary, env_vars.

if set_auth_token(base_url=env_vars.get("ues_uri"),
                  pat_token=env_vars.get("access_token"), 
                  pem_file=env_vars.get("pem_file"),
                  valid_from=int(time.time())
                 ):
    print("UES Authentication successful")
else:
    print("UES Authentication failed. Check credentials.")
    sys.exit(1)

<p style="font-size:20px; font-family:Arial"><b>3. Build the Teradata Vector Store with Teradata Package for Langchain</b></p>
<p style="font-size:18px; font-family:Arial"><b>3.1. Define the content of the page of an unstructured policy documents.</p>

In [ ]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Loyalty Discount Policy (v2, effective 2025-10-01): A business is eligible for a 10% loyalty discount if they are Active, Enterprise, and have at least 12 months of tenure."),
    Document(page_content="Exclusions: business with risk_flag = 'Y' are not eligible for loyalty discounts, regardless of tenure or business type."),
    Document(page_content="Regional exception (effective 2025-10-01): SMB Businesses in EMEA are not eligible for loyalty discounts."),
    Document(page_content="Government Businesses: Gov accounts are never auto-approved for discounts. They require manual approval and a compliance review ticket."),
    Document(page_content="Tenure definition: Tenure is measured as months between contract start_date and today. If start_date is less than 12 months ago, the Business is not eligible."),
    Document(page_content="Audit requirement: Any eligibility answer must include the policy version used and which exclusion checks were applied.")
]

<p style="font-size:18px; font-family:Arial">
  <b>3.2. Create the Teradata Vector Store</b>
</p>

<p style="font-size:16px; font-family:Arial">
  <code>TeradataVectorStore</code> is part of the <code>langchain-teradata</code> package and enables seamless integration between LangChain and Teradata Enterprise Vector Store.
</p>

<p style="font-size:16px; font-family:Arial">
  It supports three types of embedding objects:
</p>

<p style="font-size:16px; font-family:Arial">
• String identifiers (e.g., <code>"amazon.titan-embed-text-v2:0"</code>)<br>
• TeradataAI embedding objects<br>
• LangChain embedding objects (any LangChain-compatible embedding model instance)
</p>

<p style="font-size:16px; font-family:Arial">
  The <code>from_documents()</code> method provides one of the simplest ways to create a vector store. 
  You pass in your documents and an embedding model, and <code>TeradataVectorStore</code> automatically handles embedding generation, indexing, and storage.
</p>

<p style="font-size:16px; font-family:Arial">
  <b>Reference:</b><br>
  <a href="https://docs.langchain.com/oss/python/integrations/vectorstores/teradata" target="_blank">
    Teradata Vector Store – LangChain Documentation
  </a>
</p>

<p style="font-size:16px; font-family:Arial">
  Teradata Vector Store provides a variety of embedding models optimized for different environments and deployment scenarios. 
  You can review the available options here:<br>
  <a href="https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Vector-Store-User-Guide/Introduction-to-the-Enterprise-Vector-Store-User-Guide/Teradata-Vector-Store-Concepts" target="_blank">
    Teradata Vector Store Embedding Models – Teradata Documentation
  </a>
</p>



In [ ]:
from langchain_teradata import TeradataVectorStore

# Build a unique vector store name using the current database username
try:
    vs_name = env_vars.get("username")+"_"+"business_loyalty_discounts"
    
# Attempt to reference an existing Vector Store by name
# If it does not exist yet, we will create it in the loop below
    vs = TeradataVectorStore(name=vs_name,)
except Exception as e:
    print("Details: ", e)
    
# Poll for Vector Store status until it exists and is fully initialized
while True:
    df = vs.status()
    if df is None:
        print("The Teradata VectorStore doesn't exist. Creating it now.")
     
        # Create the Vector Store from documents
        # - documents: list of LangChain Document objects
        # - embedding: built-in embedding model available in Teradata Vector Store by default 
        # The embedding model will generate vectors and index the content automatically
        vs = TeradataVectorStore.from_documents(
            name=vs_name,
            documents=docs,
            embedding='amazon.titan-embed-text-v2:0' #amazon.titan-embed-text-v1 also available in EVS
            )
        print("Vector store is being created!")
        time.sleep(10) #We need a few seconds before the next status check.
    else:
        df = vs.status()
        print(f"Current status: {df}. Waiting 10 seconds...")
        time.sleep(10)
        if df is not None:
            break

print(f"The Vector Store Database already exist or has been successfully created!")    


In [ ]:
vs.get_details()

In [ ]:
vs.get_indexes_embeddings()

<p style="font-size:18px; font-family:Arial"><b>3.3 Load Structured Business data</b></p>

<p style="font-size:16px; font-family:Arial">
To evaluate loyalty discount eligibility for a Business, the agent needs business data (tenure, status, type, region, risk).  
In this step, we use a <code>buisness</code> table of businesss with sample rows designed to demonstrate common policy edge cases.
</p>

<p style="font-size:16px; font-family:Arial">
<b>💼 Use Case Summary:</b><br>
We will answer: <i>“Is a specific business eligible for a loyalty discount, and why?”</i><br>
This requires combining policy rules (from the vector store) with business attributes from the table.
</p>

<p style="font-size:16px; font-family:Arial">
<b>What’s in the <code>businesss</code> table?</b>
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.8;">
  <li><b>business_type</b> – Enterprise / SMB / Gov (drives eligibility and exceptions)</li>
  <li><b>region</b> – NA / EMEA (used for regional exceptions)</li>
  <li><b>status</b> – Active / Suspended (eligibility requires Active)</li>
  <li><b>risk_flag</b> – Y/N (hard exclusion when Y)</li>
  <li><b>start_date</b> – used to compute tenure in months</li>
</ul>

<p style="font-size:16px; font-family:Arial">
<b>Why this sample data matters:</b> each row demonstrates a different branch of the policy logic (eligible, excluded by risk, excluded by region, requires manual approval, etc.).
</p>



In [ ]:
businesses = DataFrame(in_schema("DEMO_Financial", "Business"))
businesses

<p style="font-size:20px; font-family:Arial">
  <b>4. Build the Agent with AgentStack Components: Teradata Enterprise MCP and AgentBuilder (Pro-Code)</b>
</p>

<p style="font-size:16px; font-family:Arial">
In this section, we move from data preparation to agent development using <b>AgentStack build components</b>. 
We will use <b>AgentBuilder (pro-code) with the LangChain framework</b> to define the agent’s reasoning and orchestration logic, and <b>Teradata Enterprise MCP</b> to securely connect that agent to enterprise data and governed tools.
<br><br>
For this notebook, we simulate <b>Teradata Enterprise MCP</b> using the community version of the MCP server.
</p>
<p style="font-size:18px; font-family:Arial">
  <b>4.1 Configure Custom Tools via MCP</b>
</p>

<p style="font-size:16px; font-family:Arial">
 The Teradata MCP enables admins and data engineering teams to quickly build AI interfaces tailored to specific business domains. Custom tools, prompts, cubes, and related objects are declaratively defined in YAML configuration files, allowing teams to control what an agent can access, execute, and explain—without changing application code.
</p>

<p style="font-size:16px; font-family:Arial">
In enterprise environments, these governance definitions align with and are enforced through existing <b>authentication, authorization, and RBAC systems</b>. In this notebook, YAML-based configuration is used to show what the agent is allowed to access and do, so the focus stays on governance, auditability, and least-privilege design. We are not demoing deployment patterns. 
<br>
</p>

<p style="font-size:16px; font-family:Arial">
In this demo we will show how Teradata MCP lets us expose <b>pre-approved SQL</b> as named tools.<br>
This is where the agent can call these tools and execute the pre-approved SQL. 
</p>

<p style="font-size:16px; font-family:Arial">
Run the next 3 cells to create a local <code>mcp_config/</code> folder and write two files for this demo:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px;">
  <li><code>profiles.yml</code>: which tools the agent is allowed to use</li>
  <li><code>finance_objects.yml</code>: the governed SQL tools (approved queries)</li>
</ul>

<b>Reference:</b>
<a href="https://github.com/Teradata/teradata-mcp-server/blob/main/docs/server_guide/CUSTOMIZING.md" target="_blank">
Teradata MCP Server – Customizing Guide
</a>

In [ ]:
CONFIG_DIR = "/home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config"

os.makedirs(CONFIG_DIR, exist_ok=True)

print("mcp_config directory ready")

<p style="font-size:18px; font-family:Arial"><b>4.2. Define the “approved SQL” tool</b></p>

<p style="font-size:16px; font-family:Arial">
  We create a <code>finance_objects.yml</code> file to define the custom tools exposed to the agent.
  In this demo, we define <code>finance_check_business_loyalty_eligibility</code>, a tool with
  pre-defined SQL that returns only the business facts required for the policy decision:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li>status, type, region, risk flag</li>
  <li>tenure (computed from <code>start_date</code> and <code>CURRENT_DATE</code>)</li>
  <li><code>CURRENT_DATE</code> is retrieved from the database and is not stored as a table column</li>
</ul>

<p style="font-size:16px; font-family:Arial">
  The agent uses this output to evaluate if a business is eligible and why or why not without generating SQL.
</p>

In [ ]:
%%writefile /home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config/finance_objects.yml
finance_check_business_loyalty_eligibility:
  type: tool
  description: Return business attributes needed to evaluate loyalty discount eligibility (status, type, region, risk, tenure).
  parameters:
    business_name:
      type: string
      description: Exact business_name as stored in DEMO_Financial.Business.
      required: true
  sql: |
    SELECT
      business_id,
      business_name,
      business_type,
      region,
      status,
      risk_flag,
      ((EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM start_date)) * 12
        + (EXTRACT(MONTH FROM CURRENT_DATE) - EXTRACT(MONTH FROM start_date))) AS tenure_months
    FROM "DEMO_Financial"."Business"
    WHERE business_name = :business_name;


<p style="font-size:18px; font-family:Arial"><b>4.3. Define Profiles</b></p>

<p style="font-size:16px; font-family:Arial">
  We create a <code>profiles.yml</code> file with a <code>finance_analyst</code> profile to control which MCP tools are available to the agent.
  This profile explicitly exposes the custom tool we built and restricts everything else.
</p>

<p style="font-size:16px; font-family:Arial">
  In this demo, the profile limits the agent to:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li>The approved SQL tool <code>finance_check_business_loyalty_eligibility</code> for business eligibility checks</li>
  <li>The Teradata Vector Store tool module (<code>tdvs_*</code>) which exposes all tools available to interact with the Vector Store.
</ul>

<p style="font-size:16px; font-family:Arial">
This approach enforces least-privilege access by ensuring the agent can only call explicitly approved tools.
</p>

<p style="font-size:16px; font-family:Arial">
To see examples of custom prompts, cubes, and other configurable objects, review:
<br>
<a href="https://github.com/Teradata/teradata-mcp-server/blob/main/docs/server_guide/CUSTOMIZING.md" target="_blank">
Teradata MCP Server – Customizing Guide
</a>
</p>

In [ ]:
%%writefile /home/jovyan/JupyterLabRoot/VantageCloud_Lake/UseCases/Governed_Langchain_Teradata_Agent/mcp_config/profiles.yml
finance_analyst:
  tool:
    - tdvs_*
    - finance_check_business_loyalty_eligibility
  prompt: []
  resource: []



<p style="font-size:18px; font-family:Arial"><b>4.4. Start MCP and load tools for agent use</b></p>

<p style="font-size:16px; font-family:Arial">
  We use LangChain’s <code>MultiServerMCPClient</code> to start the Teradata MCP  via <code>stdio</code> transport and load only the tools we want to expose to the agent, including custom tools.
  By restricting the tool set instead of loading the full MCP catalog, we keep the agent’s context focused on the task, reduce unnecessary token usage, and improve reliability.
  Each MCP tool is automatically converted into a callable LangChain tool, such as <code>finance_check_business_loyalty_eligibility</code>, which the agent can select at runtime.
</p>


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

TD_HOST = env_vars["host"]
TD_USER = env_vars["username"]
TD_PASSWORD = env_vars["my_variable"]  
TD_DB = TD_USER

DATABASE_URI = f"teradata://{TD_USER}:{TD_PASSWORD}@{TD_HOST}:1025/{TD_DB}"

TD_BASE_URL = env_vars.get("ues_uri")
TD_PAT = env_vars.get("access_token")
TD_PEM = env_vars.get("pem_file")

mcp_env = {
    "DATABASE_URI": DATABASE_URI,
    "TD_BASE_URL": TD_BASE_URL,
    "TD_PAT": TD_PAT,
    "TD_PEM": TD_PEM,
    "MCP_TRANSPORT": "stdio",
}


client = MultiServerMCPClient(
    {
        "teradata": {
            "transport": "stdio",
            "command": sys.executable,
            "args": [
                "-m", "teradata_mcp_server",
                "--profile", "finance_analyst",
                "--config_dir", CONFIG_DIR
            ],
            "env": mcp_env,
            "cwd": CONFIG_DIR,
        }
    }
)

mcp_tools = await client.get_tools()
print([t.name for t in mcp_tools])



<p style="font-size:16px; font-family:Arial">
After running the above cell to start the MCP Server, you should see the following tools loaded:
</p>
<p style="line-height: 1.1; font-size:16px; font-family:Arial; padding-left: 2em;">
<code style="padding:0; line-height:1.1;">
['tdvs_ask', 'tdvs_create', 'tdvs_destroy', 'tdvs_get_details', 
'tdvs_get_health', 'tdvs_grant_user_permission', 'tdvs_list', 
'tdvs_revoke_user_permission', 'tdvs_similarity_search', 
'tdvs_update', 'finance_check_business_loyalty_eligibility']
</code>
</p>

<p style="font-size:16px; font-family:Arial">
Here’s what each tool does:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li><code>tdvs_ask</code> – Performs question answering over documents stored in the Teradata Vector Store.</li>
  <li><code>tdvs_similarity_search</code> – Retrieves semantically similar documents based on a query.</li>
  <li><code>tdvs_create</code> – Creates a new vector store.</li>
  <li><code>tdvs_update</code> – Updates an existing vector store configuration or data.</li>
  <li><code>tdvs_destroy</code> – Deletes a vector store.</li>
  <li><code>tdvs_get_details</code> – Retrieves metadata and configuration details for a vector store.</li>
  <li><code>tdvs_list</code> – Lists available vector stores.</li>
  <li><code>tdvs_get_health</code> – Checks the health and availability of the vector store .</li>
  <li><code>tdvs_grant_user_permission</code> – Grants access permissions to a user for a vector store.</li>
  <li><code>tdvs_revoke_user_permission</code> – Revokes a user’s access permissions from a vector store.</li>
  <li><code>finance_check_business_loyalty_eligibility</code> – Our custom tool that runs a pre-approved SQL query to validate business loyalty discount eligibility.</li>
</ul>

<p style="font-size:16px; font-family:Arial">
Let's test running the <code>tdvs_ask</code> and <code>tdvs_similarity_search</code> . </p>


In [ ]:
import json

ask = next(t for t in mcp_tools if t.name == "tdvs_ask")
res = await ask.ainvoke({
  "vs_name": vs_name,
  "vs_ask": {"question": "Loyalty discount eligibility requirements and exclusions"}
})


payload_str = res[0]["text"]
payload = json.loads(payload_str)
print(json.dumps(payload, indent=2))



In [ ]:
import json

similarity_search = next(t for t in mcp_tools if t.name == "tdvs_similarity_search")

res = await similarity_search.ainvoke({
  "vs_name": vs_name,
  "vs_similaritysearch": {"question": "Loyalty discount eligibility requirements and exclusions"}
})

payload_str = res[0]["text"]
payload = json.loads(payload_str)
print(json.dumps(payload, indent=2))



<p style="font-size:20px; font-family:Arial"><b>5. Initialize the LLM </b></p>


In [ ]:
from langchain.chat_models import init_chat_model
llm_key = env_vars.get("litellm_key")
llm_url = env_vars.get("litellm_base_url")

llm = init_chat_model(
    model="openai-gpt-41",
    model_provider="openai",
    base_url=llm_url,
    api_key=llm_key,
)

<p style="font-size:20px; font-family:Arial"><b>6. Create the Agent</b></p>

<p style="font-size:16px; font-family:Arial">
Now we define our agent using <b>LangChain (AgentBuilder – Pro-Code Framework)</b>. 
We pass in:
</p>

<ul style="font-size:16px; font-family:Arial; margin-left:20px; line-height:1.6;">
  <li>The LLM we initialized above</li>
  <li>The governed tools loaded from Teradata MCP</li>
  <li>A custom system prompt that enforces policy-aware reasoning and tool usage rules</li>
</ul>

<p style="font-size:16px; font-family:Arial">
The agent can only act through the approved MCP tools we exposed. 
The system prompt enforces strict behavior: it must call 
<code>tdvs_similarity_search</code> to retrieve policy context from the Vector Store, 
must call <code>finance_check_business_loyalty_eligibility</code> to query business facts, 
and must never generate SQL directly.
</p>


In [ ]:
from langchain.agents import create_agent

# Note:
# tdvs_similarity_search can be swapped with tdvs_ask in prompt depending on whether
# you want retrieval-only context (similarity_search) or summarized Q&A (ask).

agent = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt=(
        "You are a compliance-aware finance assistant.\n"
        "RULES (must follow):\n"
        f"1) You MUST call tdvs_similarity_search with vs_name='{vs_name}' before making any eligibility decision.\n"
        "2) You MUST call finance_check_business_loyalty_eligibility.\n"
        "3) You may ONLY use policy clauses from tdvs_similarity_search output.\n"
        "4) If tdvs_similarity_search was not called, say you cannot answer.\n"
        "5) If SQL returns 0 rows, ask for exact business_name.\n"
        "Never generate SQL.\n"
    ))



<p style="font-size:16px; font-family:Arial">
At this point, the agent <b>can't generate or run other SQL</b>. It is constrained to the tools we explicitly provided.
<p style="font-size:16px; font-family:Arial">
Because no other SQL tools are exposed, the agent can’t execute anything outside these approved operations.
</p>



<hr style="height:2px;border:none">

<b style="font-size:20px;font-family:Arial">7. Create the Chatbot Interface</b>

<p style="font-size:16px;font-family:Arial">
The chatbot uses Panel’s <code>ChatInterface</code> to provide an interactive user experience for engaging with the governed agent.
This interface allows users to submit questions and view responses in real time, making it easy to explore how the agent reasons
over policy documents and structured business data.
</p>

<p style="font-size:16px;font-family:Arial">
At this point, we have built an agent with a very specific purpose. The agent acts as a compliance-aware assistant for a finance team,
helping determine whether a specific business qualifies for a loyalty discount and clearly explaining why.
</p>

<p style="font-size:16px;font-family:Arial">
You can ask natural-language questions such as:
</p>

<ul style="font-size:16px;font-family:Arial">
    <li>Does Acme Co qualify for a loyalty discount?</li>
    <li>Is Evergreen eligible for a loyalty discount, and why?</li>
    <li>Why does Delta Corp not qualify for a loyalty discount?</li>
    <li>Bluebird Inc</li>
</ul>


In [ ]:
pn.extension()

def _clean_user_input(contents: str) -> str:
    text = (contents or "").strip()
    if not text:
        return ""
    if "?" not in text and len(text.split()) <= 3:
        return f"Does business {text} qualify for a loyalty discount, and why?"
    return text

def _extract_answer(result):
    msgs = result.get("messages", [])
    final_answer = None
    tool_lines = []

    for m in msgs:
        t = getattr(m, "type", "")
        if t == "tool":
            name = getattr(m, "name", "tool")
            content = m.content
            tool_lines.append(f"\n--- TOOL: {name} ---\n{content}")
        elif t == "ai" and isinstance(m.content, str) and m.content.strip():
            final_answer = m.content.strip()

    return final_answer or "(No final answer returned.)", "\n".join(tool_lines)

async def _run_agent(user_text: str):
    question = _clean_user_input(user_text)
    if not question:
        return "Please enter a business name (e.g., Evergreen) or a full question."
    result = await agent.ainvoke({"messages": [HumanMessage(content=question)]})
    final_answer, tool_trace = _extract_answer(result)

    SHOW_TOOL_TRACE = True

    if SHOW_TOOL_TRACE:
        return f"{final_answer}\n\n🔎 Tool trace:\n{tool_trace}"
    else:
        return final_answer

def callback(contents, user, instance):
 
    try:
        return asyncio.run(_run_agent(contents))
    except RuntimeError:
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(_run_agent(contents))
    

In [ ]:
chat = pn.chat.ChatInterface(
    callback=callback,
    show_rerun=False,
    show_undo=False,
    callback_exception='verbose',
    show_clear=True,
    width=850,
    height=450
)

chat.servable()
chat

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><i><b>Note:</b> If the Chatbot interface isn't loading, please reload the page by clicking the <b>Reload</b> or <b>Refresh</b> button or pressing F5 on your keyboard for <b>first-time only</b> This will update the notebook with the latest modifications, and you'll be able to interact with the Chatbot using the new libraries.</i></p></div>

<p style="font-size:20px; font-family:Arial"><b>8.Run a direct query to confirm Agent Answers</b></p>

<p style="font-size:16px; font-family:Arial;">
You can  validate the agent’s answer by running a direct SQL query using the same business name provided in the user question. This query returns the authoritative business facts stored in Teradata. These values should match the facts the agent used when applying the loyalty discount policy rules:
</p>
<ul style="line-height: 1.1; font-size:16px; font-family:Arial; margin-top:0;">
  <li><code>business_type</code></li>
  <li><code>status</code></li>
  <li><code>region</code></li>
  <li><code>risk_flag</code></li>
  <li><code>tenure_months</code></li> 
</ul>

In [ ]:

business_name = input(
    "Enter the business name to evaluate for a loyalty discount "
    "(Evergreen, Bluebird Inc, Delta Corp, Acme Co, Canyon LLC, Fjord Labs):\n> "
).strip()

result = execute_sql(f"""
    SELECT
        business_id,
        business_name,
        business_type,
        region,
        status,
        risk_flag,
        /* tenure in months */
        ((EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM start_date)) * 12
          + (EXTRACT(MONTH FROM CURRENT_DATE) - EXTRACT(MONTH FROM start_date))) AS tenure_months
    FROM DEMO_Financial.Business
    WHERE business_name = '{business_name}';
""")

for row in result:
    print(row)




<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>9. Cleanup</b>
<p style = 'font-size:16px;font-family:Arial'>Call the destroy() method of the VS object to clean up the objects created during this demo. Creating a dataframe from the result of the <code>vs.status()</code> call is a simple way to check for the existance of the Vector Store.</p>

In [ ]:
while True:
    df = vs.status()
    if df is None:
        break
    else:
        vs.destroy()
        print(f"Current status: {df}. Waiting 10 seconds...")
        time.sleep(10)
        df = vs.status()

print(f"The Vector Store Database has been successfully destroyed!")

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2026. All Rights Reserved
        </div>
    </div>
</footer>